In [4]:
import torch
import wandb

import sys
sys.path.append('/home/ubuntu/taker/src')

from taker import Model
from taker.activations import get_midlayer_activations
from taker.data_classes import PruningConfig, RunDataHistory
from taker.model_repos import test_model_repos
from taker.prune import prune_and_evaluate, run_pruning



Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f6f2eea81f0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f6f2e952fe0, raw_cell="import torch
import wandb

import sys
sys.path.app.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B147.189.194.88/home/ubuntu/taker/examples/aisc-peak-activations/peak_scoring.ipynb#W0sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f6f2eea81f0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f6f2e9521a0, execution_count=4 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f6f2e952fe0, raw_cell="import torch
import wandb

import sys
sys.path.app.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B147.189.194.88/home/ubuntu/taker/examples/aisc-peak-activations/peak_scoring.ipynb#W0sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [5]:
def get_bucket_peaks(activations):
    # Check if the activations tensor is of type torch.float16
    if activations.dtype == torch.float16:
        # Convert to torch.float32 for histogram calculation
        activations_float32 = activations.float()
    else:
        # Use the original tensor if it's already in a supported data type
        activations_float32 = activations

    # Prepare for histogram computation
    min_val = activations_float32.min()
    max_val = activations_float32.max()
    bins = 100

    # Initialize an empty tensor to hold the peak values
    peak_values_float32 = torch.empty(activations_float32.size()[:-1], device=activations_float32.device, dtype=torch.float32)

    # Compute the histogram and find the peak for each neuron in every layer
    for i in range(activations_float32.size()[0]):  # Assuming the first dimension is layers
        for j in range(activations_float32.size()[1]):  # Assuming the second dimension is neurons
            hist = torch.histc(activations_float32[i, j], bins=bins, min=min_val, max=max_val)
            peak_bin = hist.argmax()
            # Compute the center value of the peak bin
            bin_width = (max_val - min_val) / bins
            peak_value = min_val + bin_width * (peak_bin.float() + 0.5)
            peak_values_float32[i, j] = peak_value

    # If the original tensor was torch.float16, convert the result back to torch.float16
    if activations.dtype == torch.float16:
        peak_values = peak_values_float32.half()
    else:
        peak_values = peak_values_float32

    return peak_values

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f6f2eea81f0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f704c59fb50, raw_cell="def get_bucket_peaks(activations):
    # Check if .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B147.189.194.88/home/ubuntu/taker/examples/aisc-peak-activations/peak_scoring.ipynb#W1sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f6f2eea81f0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f704c3e7370, execution_count=5 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f704c59fb50, raw_cell="def get_bucket_peaks(activations):
    # Check if .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B147.189.194.88/home/ubuntu/taker/examples/aisc-peak-activations/peak_scoring.ipynb#W1sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [6]:
c = PruningConfig("nickypro/tinyllama-15m",
    attn_mode="pre-out", do_attn_mean_offset=False, use_accelerator=False,
    ff_frac=0.1, attn_frac=0.1,
    token_limit=1000, focus="pile", cripple="code", wandb_entity="seperability", recalculate_activations=False,
    wandb_project="bens-tests", wandb_run_name="test notebook2", n_steps=10, scoring_normalization="peak_centered")
    #wandb_project="bens-tests", wandb_run_name="test notebook2", n_steps=10)

opt: Model = Model(c.model_repo, limit=c.token_limit, dtype="fp32",
            use_accelerator=c.use_accelerator)
focus_data = get_midlayer_activations( opt, "pile", 1e4, collect_ff=True, collect_attn=True )
cripple_data = get_midlayer_activations( opt, "code", 1e4, collect_ff=True, collect_attn=True )

# [token, layer, neuron] -> [layer, neuron, token]
focus_ff_activations   = focus_data.raw["ff"].permute( (1,2,0) )
cripple_ff_activations = cripple_data.raw["ff"].permute( (1,2,0) )
# [token, layer, attention head, attention neuron] -> [layer, attention head, attention neuron, token]
focus_attn_activations   = focus_data.raw["attn"].permute( (1,2,3,0) ).reshape( (opt.cfg.n_layers, opt.cfg.d_model, -1) )
cripple_attn_activations = cripple_data.raw["attn"].permute( (1,2,3,0) ).reshape( (opt.cfg.n_layers, opt.cfg.d_model, -1) )

focus_ff_peaks = get_bucket_peaks(focus_ff_activations).cuda()
cripple_ff_peaks = get_bucket_peaks(cripple_ff_activations).cuda()

focus_attn_peaks = get_bucket_peaks(focus_attn_activations).reshape(6, 6, 48).cuda()
cripple_attn_peaks = get_bucket_peaks(cripple_attn_activations).reshape(6, 6, 48).cuda()

# test reversing the shapes of the activatons
#print(f"ff reshaped:  + {focus_ff_activations.shape}")
#print(f"ff original:  + {focus_data.raw['ff'].shape}")
#print(f"ff peaks:  + {focus_ff_peaks.shape}")

#same thing with attn
#print(f"attn reshaped:  + {focus_attn_activations.shape}")
#print(f"attn original:  + {focus_data.raw['attn'].shape}")
#print(f"attn peaks:  + {focus_attn_peaks.shape}")

# Now get activation data again with peaks offsets
focus_data   = get_midlayer_activations( opt, "pile", 1e4, collect_ff=True, collect_attn=True, ff_peak=focus_ff_peaks, attn_peak=focus_attn_peaks )
cripple_data = get_midlayer_activations( opt, "code", 1e4, collect_ff=True, collect_attn=True, ff_peak=cripple_ff_peaks,  attn_peak=cripple_attn_peaks )

#print("focus data: ")
#print(focus_data.ff.peak_centered)
#only ff peaks
#cripple_data = get_midlayer_activations( opt, "code", 1e4, collect_ff=True, collect_attn=True, ff_peak=cripple_ff_peaks)

history = RunDataHistory(c.datasets)
wandb.init(
    project=c.wandb_project,
    entity=c.wandb_entity,
    name=c.wandb_run_name,
    )
wandb.config.update(c.to_dict(), allow_val_change=True)
with torch.no_grad():
    for i in range(c.n_steps):
        data = prune_and_evaluate(opt, c, focus_data, cripple_data, i)
        history.add(data)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f6f2eea81f0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f6f241839a0, raw_cell="c = PruningConfig("nickypro/tinyllama-15m",
    at.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B147.189.194.88/home/ubuntu/taker/examples/aisc-peak-activations/peak_scoring.ipynb#W2sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

- Loaded nickypro/tinyllama-15m
 - Registered 6 Attention Layers


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/10000.0 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (3324 > 2048). Running this sequence through the model will result in indexing errors
10048it [00:16, 612.14it/s]                            
/home/ubuntu/taker/.venv/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for codeparrot/github-code-clean contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/codeparrot/github-code-clean
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
10104it [00:17, 590.37it/s]                            


...

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Acc: 4.02|1.64 (Skip: 0.46|0.01): : 100202it [00:35, 2852.83it/s]                           


100000.0


  0%|          | 0/100000.0 [00:00<?, ?it/s]/home/ubuntu/taker/.venv/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for codeparrot/github-code-clean contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/codeparrot/github-code-clean
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Acc: 3.26|1.83 (Skip: 0.93|0.01): : 100190it [00:36, 2722.40it/s]                           
wandb: WARNING Serializing object of type ndarray that is 256112 bytes
wandb: WARNING Serializing object of type ndarray that is 256112 bytes
